## Scan Bluetooth Device

In [1]:
import asyncio
from bleak import BleakScanner

async def scan_device():
    devices = await BleakScanner.discover()
    list_devices_name = [(d.name, d.address) for d in devices if d.name and d.name != "Unknown"]
    return list_devices_name

async def find_device(list_of_target_ble_name):
    list_name = await scan_device()
    list_detected_names = [x[0] for x in list_name]

    result = True
    for target_ble_name in list_of_target_ble_name:
        if target_ble_name in list_detected_names:
            print(f"BLE device \"{target_ble_name}\" is found!")
        else:
            print(f"BLE device \"{target_ble_name}\" is NOT found!")
            result = False

    return result

async def show_device():
    list_name = await scan_device()
    for name in list_name:
        print(name)

# For Jupyter or interactive environments, just call:
await show_device()
# Or: await find_device(["MyBLEDevice1", "AnotherBLEDevice"])

# Example:
# await find_device(["OmniRing", "OmniBand"])


('Seos', '4AEF0B3C-271C-B9E2-D84D-596C7B5490DA')
('Seos', '0549B4E2-C86E-14B1-F5B3-040A34481C82')
('Seos', '9D8015AB-4BD6-44D6-4AD8-45A67718C53B')
('Seos', '7B9B5A7E-25B2-5A20-2F33-20D86DA9213A')
('[LG] webOS TV OLED42C4PUA', '465446F2-8D8D-4E07-09BA-0D17BEADB7A3')
('Seos', '886F57DB-56D7-28DC-E816-635A4BD3D561')
('PPG_Ring#1', '06D1232A-A2A5-2110-091F-DB7BDEBEE534')
('Seos', '3A6B3754-AED1-C930-0C6B-CD47E8593C49')
('Taiting’s Keys', '46522FE5-A8E6-179E-56C6-BD491F18CF5A')
('Seos', 'BAD97600-4C8D-BAD7-9A7D-D4941BD9B6EE')
('Seos', 'E854DC82-C16A-DD11-A6BC-49175B52D98D')
('Iphone15', '1989622E-8106-D0A1-CDD1-6DCF059BE6F6')
('WH-CH720N', '2DE72447-1CA6-13E5-0A4A-2C529BD8F54F')


# Capture camera

In [1]:
import cv2
import time

stop_recording = False

def make_1080p(video):
    video.set(3, 1920)
    video.set(4, 1080)

def format_elapsed_time(seconds):
    h = int(seconds // 3600)
    m = int((seconds % 3600) // 60)
    s = int(seconds % 60)
    return f"{h:02}:{m:02}:{s:02}"

def captrure_video_latency(latency,video_info):
    video = cv2.VideoCapture(0)

    if (video.isOpened() == False):
        print("Error reading video file")

    make_1080p(video)

    frame_width = int(video.get(3))
    frame_height = int(video.get(4))


    size = (frame_width, frame_height)
    print(size)

    result = cv2.VideoWriter(video_info["path"],cv2.VideoWriter_fourcc('m', 'ps', '4', 'v'),30, size)
    # result = cv2.VideoWriter('filename.avi',cv2.VideoWriter_fourcc(*'MJPG'),10, size)
    start_time = time.time()
    list_of_frame = []
    latency_secs = latency
    end_time = start_time + latency_secs
    start = False

    while time.time()<end_time:

        ret, frame = video.read()
        frame2 = cv2.flip(frame, 1)
        list_of_frame.append(frame2)

        # result.write(frame2)

        window_name = 'Frame'
        cv2.imshow(window_name, frame2)
        cv2.setWindowProperty(window_name, cv2.WND_PROP_TOPMOST, 1)
        cv2.waitKey(1)


    print("start saving")
    for fram in list_of_frame:
        result.write(fram)

    video.release()
    result.release()
    cv2.destroyAllWindows()
    cv2.waitKey(100)  # Ensure window closes properly

def captrure_video_opencv_button(video_info):
    global stop_recording
    stop_recording = False

    video = cv2.VideoCapture(1)

    if not video.isOpened():
        print("Error reading video file")
        return

    make_1080p(video)

    frame_width = int(video.get(3))
    frame_height = int(video.get(4))
    size = (frame_width, frame_height)

    result = cv2.VideoWriter(video_info["path"], cv2.VideoWriter_fourcc(*'mp4v'), 30, size)

    print("Recording... Click the red button to stop.")

    def click_event(event, x, y, flags, param):
        global stop_recording
        if event == cv2.EVENT_LBUTTONDOWN:
            if 10 <= x <= 110 and 10 <= y <= 60:  # stop button area
                print("Stop button clicked!")
                stop_recording = True

    cv2.namedWindow("Live")
    cv2.setMouseCallback("Live", click_event)

    start_time = time.time()

    while not stop_recording:
        ret, frame = video.read()
        if not ret:
            break

        frame_flipped = cv2.flip(frame, 1)

        # Display-only version of frame
        display_frame = frame_flipped.copy()

        # Draw STOP button
        cv2.rectangle(display_frame, (10, 10), (110, 60), (0, 0, 255), -1)
        cv2.putText(display_frame, "STOP", (20, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        # Draw timer
        elapsed = time.time() - start_time
        timer_text = format_elapsed_time(elapsed)
        cv2.putText(display_frame, timer_text, (frame_width - 220, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)

        # Show preview window
        cv2.imshow("Live", display_frame)

        # Save original frame (no overlay)
        result.write(frame_flipped)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Pressed 'q' to quit.")
            break

    end_time = time.time()

    video.release()
    result.release()
    cv2.destroyAllWindows()
    cv2.waitKey(100)  # Ensure window closes properly

    video_info["duration"] = end_time - start_time
    video_info["stime"] = start_time
    video_info["etime"] = end_time
    print(f"{video_info['path']} saved. Duration: {video_info['duration']:.2f} seconds.")

# Example usage
if __name__ == "__main__":
    video_info = {"name": "video", "path": "bob.mp4"}
    captrure_video_latency(5, video_info)
    # captrure_video_opencv_button(video_info)

2025-05-27 17:40:14.687 Python[87408:4069095] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


(1920, 1080)
start saving


# Capture two camera

In [ ]:
import cv2
import time
import datetime

stop_recording = False

def make_1080p(video):
    video.set(3, 1920)
    video.set(4, 1080)

def format_elapsed_time(seconds):
    h = int(seconds // 3600)
    m = int((seconds % 3600) // 60)
    s = int(seconds % 60)
    return f"{h:02}:{m:02}:{s:02}"

def capture_video_latency(latency, video_info_1, video_info_0):
    cam1 = cv2.VideoCapture(1)
    cam0 = cv2.VideoCapture(0)

    if not cam1.isOpened():
        print("Error: Camera 1 failed to open.")
        return
    if not cam0.isOpened():
        print("Error: Camera 0 failed to open.")
        return

    make_1080p(cam1)
    make_1080p(cam0)

    size1 = (int(cam1.get(3)), int(cam1.get(4)))
    size0 = (int(cam0.get(3)), int(cam0.get(4)))

    out1 = cv2.VideoWriter(video_info_1["path"], cv2.VideoWriter_fourcc(*'mp4v'), 30, size1)
    out0 = cv2.VideoWriter(video_info_0["path"], cv2.VideoWriter_fourcc(*'mp4v'), 30, size0)

    print(f"Recording both cameras for {latency} seconds...")

    start_time = time.time()
    end_time = start_time + latency

    list1 = []
    list0 = []

    while time.time() < end_time:
        ret1, frame1 = cam1.read()
        ret0, frame0 = cam0.read()

        if ret1:
            frame1 = cv2.flip(frame1, 1)
            list1.append(frame1)
            cv2.imshow("Camera 1", frame1)

        if ret0:
            frame0 = cv2.flip(frame0, 1)
            list0.append(frame0)
            cv2.imshow("Camera 0", frame0)

        cv2.waitKey(1)

    print("Saving videos...")

    for frame in list1:
        out1.write(frame)
    for frame in list0:
        out0.write(frame)

    cam1.release()
    cam0.release()
    out1.release()
    out0.release()
    cv2.destroyAllWindows()
    cv2.waitKey(100)  # Ensure window closes properly
    
def capture_video_opencv_button(video_info_1, video_info_0):
    global stop_recording
    stop_recording = False

    cam1 = cv2.VideoCapture(1)
    cam0 = cv2.VideoCapture(0)

    if not cam1.isOpened():
        print("Error: Camera 1 failed to open.")
        return
    if not cam0.isOpened():
        print("Error: Camera 0 failed to open.")
        return

    make_1080p(cam1)
    make_1080p(cam0)

    size1 = (int(cam1.get(3)), int(cam1.get(4)))
    size0 = (int(cam0.get(3)), int(cam0.get(4)))

    out1 = cv2.VideoWriter(video_info_1["path"], cv2.VideoWriter_fourcc(*'mp4v'), 30, size1)
    out0 = cv2.VideoWriter(video_info_0["path"], cv2.VideoWriter_fourcc(*'mp4v'), 30, size0)

    def click_event(event, x, y, flags, param):
        global stop_recording
        if event == cv2.EVENT_LBUTTONDOWN and 10 <= x <= 110 and 10 <= y <= 60:
            print("STOP button clicked!")
            stop_recording = True

    cv2.namedWindow("Camera 1")
    cv2.namedWindow("Camera 0")
    cv2.setMouseCallback("Camera 1", click_event)
    cv2.setMouseCallback("Camera 0", click_event)

    print("Recording... Click the red STOP button to end.")
    start_time = time.time()

    while not stop_recording:
        ret1, frame1 = cam1.read()
        ret0, frame0 = cam0.read()

        if ret1:
            flipped1 = cv2.flip(frame1, 1)
            display1 = flipped1.copy()
            cv2.rectangle(display1, (10, 10), (110, 60), (0, 0, 255), -1)
            cv2.putText(display1, "STOP", (20, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            elapsed = time.time() - start_time
            cv2.putText(display1, format_elapsed_time(elapsed), (size1[0] - 220, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)
            cv2.imshow("Camera 1", display1)
            out1.write(flipped1)

        if ret0:
            flipped0 = cv2.flip(frame0, 1)
            display0 = flipped0.copy()
            cv2.rectangle(display0, (10, 10), (110, 60), (0, 0, 255), -1)
            cv2.putText(display0, "STOP", (20, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            elapsed = time.time() - start_time
            cv2.putText(display0, format_elapsed_time(elapsed), (size0[0] - 220, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)
            cv2.imshow("Camera 0", display0)
            out0.write(flipped0)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Pressed 'q' to quit.")
            break

    end_time = time.time()

    cam1.release()
    cam0.release()
    out1.release()
    out0.release()
    cv2.destroyAllWindows()

    video_info_1["duration"] = video_info_0["duration"] = end_time - start_time
    print(f"Saved: {video_info_1['path']} and {video_info_0['path']}")

# Example usage
if __name__ == "__main__":
    timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    video_info_1 = {"name": "cam1", "path": f"cam1_{timestamp}.mp4"}
    video_info_0 = {"name": "cam0", "path": f"cam0_{timestamp}.mp4"}
    
    # For fixed-duration capture
    capture_video_latency(5, video_info_1, video_info_0)

    # For button-based recording
    # capture_video_opencv_button(video_info_1, video_info_0)


Recording both cameras for 5 seconds...
Saving videos...


# Connect one imu device only

In [ ]:
import asyncio
from bleak import BleakClient
import struct
import time
import threading
import queue
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import json
import os

# Ensure that matplotlib uses the appropriate backend
matplotlib.use("TkAgg")

# Constants
UART_RX_UUID = "6e400003-b5a3-f393-e0a9-e50e24dcca9e"  # Nordic NUS characteristic for RX
TIME_LATENCY = 1200  # Duration to collect data in seconds
DATA_DISPLAY = [2, 12, 13, 14]  # Indices of data to display
TOTAL_DISPLAY = 300  # Number of data points to display on the plot
FREQ_NUM_AVG = 100  # Number of timestamps to use for frequency calculation


class BLEDataCollector:
    def __init__(self, address, latency, data_display, total_display, freq_num_avg, parent_path, iot_info):
        """
        Initialize the BLEDataCollector with necessary configurations.

        Args:
            address (str): BLE device address.
            latency (int): Duration to collect data in seconds.
            data_display (list): Indices of data to display.
            total_display (int): Number of data points to display on the plot.
            freq_num_avg (int): Number of timestamps to use for frequency calculation.
            parent_path (str): Directory path to save data.
            iot_info (dict): Dictionary containing device information.
        """
        # Configuration
        self.address = address
        self.latency = latency
        self.data_display = data_display
        self.total_display = total_display
        self.freq_num_avg = freq_num_avg
        self.parent_path = parent_path
        self.iot_info = iot_info  # Renamed from imu_info to iot_info

        # Initialize variables
        self.start_time = None
        self.end_time = None
        self.count = 0
        self.ble_connected = False
        self.display_loop_finished = threading.Event()  # Event to signal when to close the plot

        # Queues for thread-safe communication
        self.data_queue = queue.Queue()       # For plotting
        self.write_queue = queue.Queue()      # For file writing

        # Initialize plot data
        self.y_data = {i: [] for i in self.data_display}

        # Start the writer thread
        self.writer_thread = threading.Thread(target=self.writer, daemon=True)
        self.writer_thread.start()

    def unpack_float(self, byte_data):
        """Unpack 4 bytes into a float."""
        return struct.unpack('f', byte_data)[0]

    def decode_byte_data(self, byte_data):
        """
        Decode byte data into a list of floats.

        Args:
            byte_data (bytes): Incoming byte data from BLE device.

        Returns:
            list: List of decoded float values.
        """
        float_array = []
        for i in range(0, len(byte_data), 4):
            if i + 4 <= len(byte_data):
                tmp_float = self.unpack_float(byte_data[i:i+4])
                float_array.append(tmp_float)
            else:
                # Handle incomplete data
                print(f"Incomplete data received: {byte_data[i:]}")
        return float_array

    def callback(self, sender, data):
        """
        Callback function for BLE notifications.

        Args:
            sender (str): The handle of the characteristic.
            data (bytes): The data received from the BLE device.
        """
        self.count += 1
        # print(f"Callback called. Notification #{self.count}")  # Debug statement
        current_time = time.time()
        if self.count == 1:
            self.start_time = current_time
            print("Data collection started.")
        else:
            self.end_time = current_time

        result = self.decode_byte_data(data)
        # print(f"Data received: {result}")
        timestamp = time.time()
        result.append(timestamp)

        # Put data into the queues
        self.data_queue.put(result)
        self.write_queue.put(result)
        # print(f"Data enqueued for plotting and writing: {result}")

    def get_avg_freq(self, timestamps):
        """
        Calculate average frequency from timestamps.

        Args:
            timestamps (list): List of timestamp values.

        Returns:
            float: Average frequency in Hz.
        """
        if len(timestamps) >= 2:
            duration = timestamps[-1] - timestamps[0]
            if duration > 0:
                return len(timestamps) / duration
        return 0

    async def connect_to_device(self):
        """
        Connect to the BLE device and collect data.

        This coroutine handles the BLE connection, subscribes to notifications,
        and collects data for the specified latency duration.
        """
        self.ble_connected = True
        try:
            async with BleakClient(self.address) as client:
                print(f"Connected to {self.iot_info['name']}")  # Updated reference
                # print(f"Subscribing to notifications on {UART_RX_UUID}")
                await client.start_notify(UART_RX_UUID, self.callback)
                # print("Notification subscription successful.")
                await asyncio.sleep(self.latency)
                await client.stop_notify(UART_RX_UUID)
                print("Stopped notifications.")
        except Exception as e:
            print(f"An error occurred during BLE communication: {e}")
        finally:
            self.ble_connected = False
            self.display_loop_finished.set()  # Signal that data collection is finished

    def run_ble_loop(self):
        """Run the BLE event loop in a separate thread."""
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        loop.run_until_complete(self.connect_to_device())
        loop.close()

    def writer(self):
        """
        Writer thread that writes data from write_queue to file.

        This thread continuously listens to the write_queue and writes incoming
        data to the designated file immediately upon receipt.
        """
        # Wait until start_time is set (i.e., data collection has started)
        while self.start_time is None:
            time.sleep(0.1)

        # Generate file path based on start_time
        data_dir = os.path.join(self.parent_path, self.iot_info['user_name'], 'data')
        os.makedirs(data_dir, exist_ok=True)

        timestamp_str = time.strftime("%Y%m%d_%H%M%S", time.localtime(self.start_time))
        data_file_name = f"{self.iot_info['name']}_{timestamp_str}.txt"
        self.data_file_path = os.path.join(data_dir, data_file_name)

        print(f"Data will be written to: {self.data_file_path}")  # Debug statement

        try:
            with open(self.data_file_path, 'w') as f:
                while True:
                    try:
                        # Wait for data for up to 1 second
                        data = self.write_queue.get(timeout=1)
                        f.write(','.join(map(str, data)) + '\n')
                        f.flush()  # Ensure data is written immediately
                        # print(f"Data written to file: {data}")
                    except queue.Empty:
                        if self.display_loop_finished.is_set():
                            # print("Writer thread terminating: No more data to write.")
                            break
        except Exception as e:
            print(f"Failed to write data to {self.data_file_path}: {e}")


    def plot_data(self):
        """Set up and run the real-time plot."""
        fig, axs = plt.subplots(len(self.data_display), figsize=(10, 6))
        if len(self.data_display) == 1:
            axs = [axs]  # Ensure axs is always a list

        # Initialize frequency text
        fre_text = fig.text(0.01, 0.95, "Freq: -1 Hz", fontsize=12)

        # Initialize timestamps list for frequency calculation
        timestamps = []

        def update_plot(frame):
            nonlocal timestamps
            while not self.data_queue.empty():
                result = self.data_queue.get()
                timestamp = result[-1]
                timestamps.append(timestamp)
                if len(timestamps) > self.freq_num_avg:
                    timestamps = timestamps[-self.freq_num_avg:]

                # Update y_data only for available indices
                for i in self.data_display:
                    if i < len(result) - 1:
                        self.y_data[i].append(result[i])
                    else:
                        # Append NaN if data is missing
                        self.y_data[i].append(float('nan'))
                    # Trim data to TOTAL_DISPLAY points
                    if len(self.y_data[i]) > self.total_display:
                        self.y_data[i] = self.y_data[i][-self.total_display:]

            if self.ble_connected or not self.display_loop_finished.is_set():
                for ax_index, data_index in enumerate(self.data_display):
                    axs[ax_index].clear()
                    axs[ax_index].plot(self.y_data[data_index], color="blue", alpha=0.9)
                    axs[ax_index].set_ylabel(f'Data {data_index}')
                    axs[ax_index].set_xlabel("Sample Number")
                if self.start_time and self.end_time:
                    elapsed_time = self.end_time - self.start_time
                    remaining_time = int(self.latency - elapsed_time)
                    if remaining_time < 0:
                        remaining_time = 0
                else:
                    remaining_time = self.latency
                fig.suptitle(f'Time Remaining: {remaining_time}s')

                avg_freq = self.get_avg_freq(timestamps)
                fre_text.set_text(f'Freq: {avg_freq:.2f} Hz')
            else:
                plt.close()
                return

        ani = FuncAnimation(fig, update_plot, interval=100)
        plt.show()

    def run(self):
        """Run the data collection and plotting."""
        # Start BLE data collection in a separate thread
        data_thread = threading.Thread(target=self.run_ble_loop, daemon=True)
        data_thread.start()
        # print("BLE data collection thread started.")

        # Start plotting in the main thread
        self.plot_data()

        # After plotting window is closed, wait for data collection thread to finish
        data_thread.join()
        # print("BLE data collection thread joined.")

        # Signal the writer thread to finish
        self.display_loop_finished.set()
        # print("Signaled writer thread to terminate.")

        # Wait for writer thread to finish
        self.writer_thread.join()
        # print("Writer thread has terminated.")

        # Save device info
        if self.start_time:
            timestamp_str = time.strftime("%Y%m%d_%H%M%S", time.localtime(self.start_time))
        else:
            timestamp_str = "unknown_time"

        iot_info = {  # Renamed from imu_info to iot_info for clarity
            "name": self.iot_info["name"],
            "user_name": self.iot_info["user_name"],
            "address": self.iot_info["address"],
            "path": self.data_file_path,
            "duration": self.end_time - self.start_time if self.end_time and self.start_time else 0,
            "total": self.count,
            "fs": self.count / (self.end_time - self.start_time) if self.end_time and self.start_time else 0,
            "stime": self.start_time,
            "etime": self.end_time
        }

        dict_file_name = f"{self.iot_info['name']}_{timestamp_str}_info.json"
        dict_file_path = os.path.join(self.parent_path, self.iot_info['user_name'], 'data', dict_file_name)

        try:
            with open(dict_file_path, 'w') as f:
                json.dump(iot_info, f, indent=4)
            print(f"Device info saved to {dict_file_path}")
        except Exception as e:
            print(f"Failed to save device info to {dict_file_path}: {e}")


    def stop(self):
        """Stop data collection."""
        self.display_loop_finished.set()

def main():
    """
    Main function to configure and start the BLE data collection.
    """
    # Configuration
    parent_path = "./"  # Ensure this directory exists and is writable
    iot_info = {  # iot_info
        "name": "PPG_Ring#1",
        "user_name": "Taiting",
        "address": "06D1232A-A2A5-2110-091F-DB7BDEBEE534",  # Update with your BLE device's address
    }

    # Instantiate the BLEDataCollector
    collector = BLEDataCollector(
        address=iot_info["address"],
        latency=TIME_LATENCY,
        data_display=DATA_DISPLAY,
        total_display=TOTAL_DISPLAY,
        freq_num_avg=FREQ_NUM_AVG,
        parent_path=parent_path,
        iot_info=iot_info  # Updated parameter name
    )

    # Run the data collection and plotting
    collector.run()

if __name__ == "__main__":
    main()


/var/folders/6_/fhplfkl93ts3d6yt6kdrkpc00000gn/T/ipykernel_79424/2119996040.py:253: UserWarning: frames=None which we can infer the length of, did not pass an explicit *save_count* and passed cache_frame_data=True.  To avoid a possibly unbounded cache, frame data caching has been disabled. To suppress this warning either pass `cache_frame_data=False` or `save_count=MAX_FRAMES`.
  ani = FuncAnimation(fig, update_plot, interval=100)


Connected to PPG_Ring#1
Data collection started.
Data will be written to: ./Taiting/data/PPG_Ring#1_20250718_170131.txt
Stopped notifications.
Device info saved to ./Taiting/data/PPG_Ring#1_20250718_170131_info.json


: 

# connect one device with video

In [12]:
import asyncio
from bleak import BleakClient
import struct
import time
import threading
import queue
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import json
import os
import cv2

matplotlib.use("TkAgg")

UART_RX_UUID = "6e400003-b5a3-f393-e0a9-e50e24dcca9e"

class BLEDataCollector:
    def __init__(self, address, latency, data_display, total_display, freq_num_avg, parent_path, iot_info):
        self.address = address
        self.latency = latency
        self.data_display = data_display
        self.total_display = total_display
        self.freq_num_avg = freq_num_avg
        self.parent_path = parent_path
        self.iot_info = iot_info

        self.timestamp_str = str(int(time.time()))
        self.start_time = None
        self.end_time = None
        self.video_stime = None
        self.video_etime = None
        self.video_path = None
        self.sensor_file_path = None
        self.count = 0
        self.ble_connected = False
        self.display_loop_finished = threading.Event()

        self.data_queue = queue.Queue()
        self.write_queue = queue.Queue()

        self.y_data = {i: [] for i in self.data_display}

        self.writer_thread = threading.Thread(target=self.writer, daemon=True)
        self.writer_thread.start()

    def unpack_float(self, byte_data):
        return struct.unpack('f', byte_data)[0]

    def decode_byte_data(self, byte_data):
        float_array = []
        for i in range(0, len(byte_data), 4):
            if i + 4 <= len(byte_data):
                float_array.append(self.unpack_float(byte_data[i:i+4]))
            else:
                print(f"Incomplete data received: {byte_data[i:]}")
        return float_array

    def callback(self, sender, data):
        self.count += 1
        current_time = time.time()
        if self.count == 1:
            self.start_time = current_time
            print("Data collection started.")
        else:
            self.end_time = current_time

        result = self.decode_byte_data(data)
        result.append(current_time)
        self.data_queue.put(result)
        self.write_queue.put(result)

    def get_avg_freq(self, timestamps):
        if len(timestamps) >= 2:
            duration = timestamps[-1] - timestamps[0]
            return len(timestamps) / duration if duration > 0 else 0
        return 0

    def capture_video(self):
        video_file_name = f"{self.iot_info['name']}_{self.timestamp_str}.mp4"
        self.video_path  = os.path.join(self.parent_path, self.iot_info['user_name'], 'data', video_file_name)
        video_info = {"name": "video", "path": self.video_path }
        video = cv2.VideoCapture(0)
        if not video.isOpened():
            print("Error reading video file")
            return

        video.set(3, 1920)
        video.set(4, 1080)
        size = (int(video.get(3)), int(video.get(4)))
        result = cv2.VideoWriter(video_info["path"], cv2.VideoWriter_fourcc(*'mp4v'), 30, size)
        self.video_stime = time.time()
        list_of_frame = []
        while not self.display_loop_finished.is_set():
            ret, frame = video.read()
            if not ret:
                break
            frame2 = cv2.flip(frame, 1)
            list_of_frame.append(frame2)
        self.video_etime = time.time()
        print(f"Video saved to {self.video_path }")
        for fram in list_of_frame:
            result.write(fram)
        video.release()
        result.release()

    async def connect_to_device(self):
        self.ble_connected = True
        video_thread = threading.Thread(target=self.capture_video)
        video_thread.start()
        try:
            async with BleakClient(self.address) as client:
                print(f"Connected to {self.iot_info['name']}")
                await client.start_notify(UART_RX_UUID, self.callback)
                await asyncio.sleep(self.latency)
                await client.stop_notify(UART_RX_UUID)
        except Exception as e:
            print(f"BLE communication error: {e}")
        finally:
            self.ble_connected = False
            self.display_loop_finished.set()
            video_thread.join()

    def run_ble_loop(self):
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        loop.run_until_complete(self.connect_to_device())
        loop.close()

    def writer(self):
        while self.start_time is None:
            time.sleep(0.1)

        data_dir = os.path.join(self.parent_path, self.iot_info['user_name'], 'data')
        os.makedirs(data_dir, exist_ok=True)

        data_file_name = f"{self.iot_info['name']}_{self.timestamp_str}.txt"
        self.sensor_file_path = os.path.join(data_dir, data_file_name)

        print(f"Data will be written to: {self.sensor_file_path}")

        try:
            with open(self.sensor_file_path, 'w') as f:
                while True:
                    try:
                        data = self.write_queue.get(timeout=1)
                        f.write(','.join(map(str, data)) + '\n')
                        f.flush()
                    except queue.Empty:
                        if self.display_loop_finished.is_set():
                            break
        except Exception as e:
            print(f"Failed to write data to {self.sensor_file_path}: {e}")

    def plot_data(self):
        fig, axs = plt.subplots(len(self.data_display), figsize=(10, 6))
        if len(self.data_display) == 1:
            axs = [axs]
        fre_text = fig.text(0.01, 0.95, "Freq: -1 Hz", fontsize=12)
        timestamps = []

        def update_plot(frame):
            nonlocal timestamps
            while not self.data_queue.empty():
                result = self.data_queue.get()
                timestamps.append(result[-1])
                timestamps = timestamps[-self.freq_num_avg:]
                for i in self.data_display:
                    if i < len(result) - 1:
                        self.y_data[i].append(result[i])
                    else:
                        self.y_data[i].append(float('nan'))
                    self.y_data[i] = self.y_data[i][-self.total_display:]

            if self.ble_connected or not self.display_loop_finished.is_set():
                for ax_index, data_index in enumerate(self.data_display):
                    axs[ax_index].clear()
                    axs[ax_index].plot(self.y_data[data_index], color="blue", alpha=0.9)
                    axs[ax_index].set_ylabel(f'Data {data_index}')
                    axs[ax_index].set_xlabel("Sample Number")
                if self.start_time and self.end_time:
                    elapsed = self.end_time - self.start_time
                    remaining = int(self.latency - elapsed)
                else:
                    remaining = self.latency
                fig.suptitle(f'Time Remaining: {remaining}s')
                avg_freq = self.get_avg_freq(timestamps)
                fre_text.set_text(f'Freq: {avg_freq:.2f} Hz')
            else:
                plt.close()

        ani = FuncAnimation(fig, update_plot, interval=100)
        plt.show()

    def run(self):
        ble_thread = threading.Thread(target=self.run_ble_loop)
        ble_thread.start()
        self.plot_data()
        ble_thread.join()
        self.display_loop_finished.set()
        self.writer_thread.join()

        if self.start_time:
            timestamp_str = str(int(time.time()))
        else:
            timestamp_str = "unknown_time"

        iot_info = {
            "name": self.iot_info["name"],
            "user_name": self.iot_info["user_name"],
            "address": self.iot_info["address"],
            "sensor_path": self.sensor_file_path,
            "video_path": self.video_path,
            "duration": self.end_time - self.start_time if self.end_time and self.start_time else 0,
            "total": self.count,
            "fs": self.count / (self.end_time - self.start_time) if self.end_time and self.start_time else 0,
            "sensor_stime": self.start_time,
            "sensor_etime": self.end_time,
            "video_stime": self.video_stime,
            "video_etime": self.video_etime
        }

        dict_file_name = f"{self.iot_info['name']}_{self.timestamp_str}_info.json"
        dict_file_path = os.path.join(self.parent_path, self.iot_info['user_name'], 'data', dict_file_name)

        try:
            with open(dict_file_path, 'w') as f:
                json.dump(iot_info, f, indent=4)
            print(f"Device info saved to {dict_file_path}")
        except Exception as e:
            print(f"Failed to save device info to {dict_file_path}: {e}")

    def stop(self):
        self.display_loop_finished.set()

TIME_LATENCY = 10
DATA_DISPLAY = [0, 1, 2]
TOTAL_DISPLAY = 300
FREQ_NUM_AVG = 100

def main():
    parent_path = "./"
    iot_info = {
        "name": "wearable#1",
        "user_name": "demo",
        "address": "4C08C659-BA4B-E40A-E9FA-5A035112216D"
    }

    collector = BLEDataCollector(
        address=iot_info["address"],
        latency=TIME_LATENCY,
        data_display=DATA_DISPLAY,
        total_display=TOTAL_DISPLAY,
        freq_num_avg=FREQ_NUM_AVG,
        parent_path=parent_path,
        iot_info=iot_info
    )
    collector.run()

if __name__ == "__main__":
    main()


/var/folders/6_/fhplfkl93ts3d6yt6kdrkpc00000gn/T/ipykernel_75882/1271299119.py:192: UserWarning: frames=None which we can infer the length of, did not pass an explicit *save_count* and passed cache_frame_data=True.  To avoid a possibly unbounded cache, frame data caching has been disabled. To suppress this warning either pass `cache_frame_data=False` or `save_count=MAX_FRAMES`.
  ani = FuncAnimation(fig, update_plot, interval=100)


Connected to wearable#1
Data collection started.
Data will be written to: ./demo/data/wearable#1_1747984450.txt
Video saved to ./demo/data/wearable#1_1747984450.mp4
Device info saved to ./demo/data/wearable#1_1747984450_info.json
